In [4]:
import pandas as pd
from haversine import haversine

dataCoords = pd.read_csv("data/koordinaten.csv", delimiter=';')
referenzCoords = pd.read_csv("data/ganzeFahrtReferenz.tcsv", delimiter=';')

# Format Anpassung bei "TIME, Messstartpunkt wird gesetzt, und Index reset zu 0
def sortdata(dataframe):
    dataframe['TIME'] = dataframe['TIME'].str[:19]
    dataframe = dataframe.loc[dataframe["TIME"] > "2022-12-28T08:36:19"]
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

# Zeitüberlappung für genauen Messwert. Die Referenz "TIME" passt sich der von Toll-Now an
def timefiltering(dataframe):
    mask = dataframe["TIME"].isin(tollnow["TIME"])
    dataframe = dataframe[mask]
    dataframe = dataframe.reset_index(drop=True)
    return dataframe


tollnow = sortdata(dataCoords)
referenzCoords = sortdata(referenzCoords)

referenz = timefiltering(referenzCoords)

# Erstelle Subset nur mit den Werten von LAT und LON
tollNowSubset = tollnow.loc[:,["LAT", "LON", "TIME"]]
referenzSubset = referenz.loc[:,["LAT", "LON", "TIME"]]

def createPoint(row):
    point = (row["LAT"], row["LON"])
    return point

# Wende die Methode createPoint() für jede Zeile in DF an und speicher den Punkt in einer neuen Spalte 
tollNowSubset["TollNow Point"] = tollNowSubset.apply(createPoint, axis = 1)
referenzSubset["Referenz Point"] = referenzSubset.apply(createPoint, axis = 1)


def haversineFormula(row):
    result = haversine(row["TollNow Point"], row["Referenz Point"])*1000
    return result
   
# Erstelle ein neues DF mit den Werten von tollNow und referenz    
haversineDF = pd.merge(tollNowSubset, referenzSubset, on = "TIME")

# Delete the LAT and LON columns 
haversineDF = haversineDF.drop(columns = ["LAT_x", "LON_x", "LAT_y", "LON_y"])

# Wende Methode haversineFormula() für jede Zeile an, und speicher das Ergebnis in einer neuen Spalte ab 
haversineDF["Result"] = haversineDF.apply(haversineFormula, axis = 1)
 
haversineDF.to_csv('my_data.csv', index=False)

haversineDF


ModuleNotFoundError: No module named 'haversine'

In [ ]:
import numpy as np

def ortungsgenauigkeit(dataframe, quantileValue):
    series = dataframe.loc[:, "Result"]
    sortedSeries = series.sort_values()
    quantile = np.quantile(sortedSeries, quantileValue)
    return print(f"Die Ortungsgenauigkeit bei einem {quantileValue}-Quantil beträgt: ", quantile)

ortungsgenauigkeit(haversineDF, 0.99)

series = haversineDF.loc[:, "Result"]
sortedSeries = series.sort_values()

In [ ]:
def maxminvalue(column, keyword):
    if keyword == max:
        return print(f"Maximaler Wert in: {column}: ", haversineDF[f"{column}"].max())
    elif keyword == min:
        return print(f"Minimaler Wert in: {column}: ", haversineDF[f"{column}"].min())


def lenwithcondition(column, op, value):
    if op == "<":
        return print(f"Länge von {column}: ", len(haversineDF[haversineDF[column] < value]))
    elif op == ">":
        return print(f"Länge von {column}: ", len(haversineDF[haversineDF[column] > value]))


maxminvalue("Result", min)

# Anzahl von Crosstrackfehlern bei einer Abweichung von 13 Metern 
lenwithcondition("Result", ">", 13.0)

In [ ]:
# Erstelle neuen DataFrame für einzelne Spalte mit TollNow Coords und der Zeit
distanceDF = pd.DataFrame()

distanceDF["TollNow Point"] = haversineDF.loc[:, "TollNow Point"]
distanceDF["Date"] = haversineDF.loc[:, "Date"]

def applyHaversineOnPoints(dataframe):
    for index in dataframe.index:
        if index < len(dataframe) - 1:
            point1 = dataframe["TollNow Point"][index]
            index += 1
            point2 = dataframe["TollNow Point"][index]
            dataframe.loc[index, "Haversine Distance"] = haversine(point1, point2)    
            index -= 1
        else:
            break
        
applyHaversineOnPoints(distanceDF)

# Berechne mir die Summe aus den ganzen berechneten Abständen von den Punkten
def getTotalDistance():
    a = distanceDF["Haversine Distance"].sum()
    return a

totalDistance = getTotalDistance()
print(totalDistance, "km Strecke, ist der LKW insgesamt gefahren")


def changeTypeOfTime(dataframe):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])

changeTypeOfTime(distanceDF)

def getAvgSpeed():
    # Differenz von letztem Zeitstempel und erstem
    timePassed = haversineDF["Date"].max()-haversineDF["Date"].min()
    # Wandle das TimeDelta Object timePassed erst in Sekunden und dann in reine Stunden um
    hours = timePassed.total_seconds() / 3600
    # Die klassische Zeitberchnungs Formel
    a = totalDistance / hours
    return a

avgSpeed = getAvgSpeed()
print(avgSpeed, "km/h schnell, ist der LKW gefahren")

In [2]:
import pandas as pd
from haversine import haversine




dataCoords = pd.read_csv("data/koordinaten.csv", delimiter=';')

dataCoords = dataCoords.drop(columns = ["TIME", "DIRECTION", "SPEED", "TOP"])

dataCoords.to_csv("myCoords.csv", index = False)
